In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brindo/Tweets_Indonesia.csv
/kaggle/input/brindo/Tweets_Brazil.csv
/kaggle/input/abbrevsskk/Abbreviations.txt


In [2]:
!pip install gdown

In [3]:
# !gdown 1HWihiRRaesJjgeTIJewKLig57e3JXlJx

In [4]:
# df_japan = pd.read_csv("/kaggle/working/Tweets_Japan.csv")

In [5]:
# df_brazil = pd.read_csv("/kaggle/input/brindo/Tweets_Brazil.csv")

In [6]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 3.4 MB/s eta 0:00:00


In [7]:
import string
import nltk
import csv
import re
import demoji
from nltk.stem.wordnet import WordNetLemmatizer

In [8]:
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions
    
    def lower(self, tweet):
        return tweet.lower()
    
    def abbreviate(self, tweet):
        tweet = tweet.split(' ')
        j = 0
        for str_ in tweet:
            fileName = '/kaggle/input/abbrevsskk/Abbreviations.txt'
            accessMode = 'r'
            with open(fileName, accessMode) as csvfile:
                dataFromFile = csv.reader(csvfile, delimiter = '=')
                str_ = re.sub('[^a-zA-Z0-9-_.]', '', str_)
                for row in dataFromFile:
                    if str_.upper() == row[0]:
                        tweet[j] = row[1]
                csvfile.close()
            j += 1
        return ' '.join(tweet)
    
    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet
    
    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])
        
        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)
    
    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)
    
    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)
        return tweet


In [9]:
contractions = { 
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [10]:
contractionsWithAnotherInvertedComma = { 
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [11]:
from tqdm import tqdm
tqdm.pandas()

In [12]:
# pp_class = preprocess(df_brazil, contractions, contractionsWithAnotherInvertedComma)
# df_brazil['text'] = df_brazil['text'].progress_apply(lambda x : pp_class.preprocess_tweet(x))

In [13]:
hinduismWords = [
  "Hinduism",
  "Dharma",
  "Karma",
  "Yoga",
  "Meditation",
  "Mantra",
  "Puja",
  "Bhakti",
  "Ganesha",
  "Shiva",
  "Vishnu",
  "Krishna",
  "Rama",
  "Hanuman",
  "Aum",
  "Samsara",
  "Moksha",
  "Bhagavad Gita",
  "Vedas",
  "Upanishads",
  "Sanskrit",
  "Guru",
  "Avatar",
  "Aarti",
  "Atman",
  "Sutra",
  "Siddhi",
  "Swami",
  "Tantra",
  "Ahimsa",
  "Jnana",
  "Sannyasa",
  "Murti",
  "Mandala",
  "Sadhana",
  "Kundalini",
  "Brahman",
  "Arya",
  "Samskara",
  "Kirtan",
  "Yajna",
  "Kali",
  "Durga",
  "Lakshmi",
  "Saraswati",
  "Diwali",
  "Holi",
  "Navaratri",
  "Raksha Bandhan",
  "Surya",
  "Chakras",
  "Ashram",
  "Pilgrimage",
  "Namaste",
  "Sadhguru",
  "Om Namah Shivaya",
  "Bhagwan",
  "Sanatana Dharma",
  "Darshan",
  "Yagna",
  "Prana",
  "Vairagya",
  "Yajnopavita",
  "Purnima",
  "Shankha",
  "Kumbh Mela",
  "Karma Yoga",
  "Bhava",
  "Rishis",
  "Nirvana",
  "Gandharva",
  "Aryavarta",
  "Ganga",
  "Ashvattha",
  "Surya Namaskar",
  "Bhumi Puja",
  "Deeksha",
  "Yogic",
  "Yatra",
  "Bhajan",
  "Panchakarma",
  "Nadi",
  "Satyam Shivam Sundaram",
  "Raj Yoga",
  "Sadhaka",
  "Pashupati",
  "Prarabdha Karma",
  "Kirtana",
  "Sarva Dharma Sambhava",
  "Yogini",
  "Atma Jnana",
  "Akasha",
  "Purna",
  "Aham Brahmasmi",
  "Gurukula",
  "Shanti",
  "Arogya",
  "Acharya",
  "Loka Samastha Sukhino Bhavantu",
  "Dvaita",
  "Advaita",
  "Vairagi",
  "Yamuna",
  "Ashaucha",
  "Samadhi",
  "Sanyasi",
  "Prasada",
  "Ananda",
  "Brahmacharya",
  "Nyasa",
  "Sadhana Chatushtaya",
  "Swadharma",
  "Pinda",
  "Annapurna",
  "Pandit",
  "Klesha",
  "Vairagin",
  "Samsaric",
  "Gyan Mudra",
  "Bhava Samadhi",
  "Upanayana",
  "Antahkarana",
  "Yogasana",
  "Uttarayana",
  "Aparigraha",
  "Turiya",
  "Sahaja",
  "Dvija",
  "Dvadashi",
  "Yogi",
  "Diksha",
  "Sankalpa",
  "Kriya",
  "Achintya",
  "Bhagavata",
  "Jivanmukta",
  "Vipassana",
  "Paramatman",
  "Siddhartha",
  "Yagya",
  "Chitta",
  "Koshas",
  "Yuga",
  "Murti Puja",
  "Vairagya",
  "Tattva",
  "Agni",
  "Japamala",
  "Sarvam Khalvidam Brahma",
  "Vaishnavism",
  "Shaivism",
  "Shaktism",
  "Nirvikalpa",
  "Advaita Vedanta",
  "Trimurti",
  "Vairagya",
  "Tapas",
  "Yajur Veda",
  "Sama Veda",
  "Rig Veda",
  "Atharva Veda",
  "Surya Namaskar",
  "Arati",
  "Guru Purnima",
  "Yogic",
  "Namaskar",
  "Nirguna",
  "Sadhana",
  "Nishkama Karma",
  "Yogasana",
  "Kirtan",
  "Asana",
  "Akshaya Tritiya",
  "Pranayama",
  "Mandir",
  "Brahmaloka",
  "Sarva Dharma Sambhava",
  "Ardhanarishwara",
  "Ghar Wapsi",
  "Maharshi",
  "Atmanjnan",
  "Vichara",
  "Japam",
  "Bhikshu",
  "Upavasa",
  "Mouna",
  "Jnani",
  "Jnana Yoga",
  "Tapasya",
  "Bhava Samadhi",
  "Jagadguru",
  "Jivanmukta",
  "Kirtan",
  "Nigraha",
  "Swadharma",
  "Nirguna",
  "Akshaya Tritiya",
  "Bhagavata",
  "Paramahansa",
  "Vivekananda",
  "Yagya",
  "Chidananda",
  "Tattva",
  "Aratrika",
  "Abhisheka",
  "Kailasa",
  "Bhava",
  "Nitya Karma",
  "Sannyasi",
  "Brahmachari",
  "Vairagi",
  "Kumbh Mela",
  "Siddha",
  "Yajna",
  "Vrat",
  "Nyasa",
  "Namarupa",
  "Homa",
  "Prasada",
  "Yajnopavita",
  "Rishi",
  "Jiva",
  "Lila",
  "Sat-Chit-Ananda",
  "Jnana Mudra",
  "Sankalpa",
  "Akasha",
  "Chinmaya",
  "Purna",
  "Akhand",
  "Aham Brahmasmi",
  "Nirvikalpa Samadhi",
  "Gurukula",
  "Shanti",
  "Arogya",
  "Acharya",
  "Jnana Vijnana",
  "Yajur Veda",
  "Sama Veda",
  "Rig Veda",
  "Atharva Veda",
  "Soma",
  "Ashtavakra",
  "Guru Nanak",
  "Radha Krishna",
  "Kundalini Yoga",
  "Vedanta",
  "Prana",
  "Gurukul",
  "Yugadharma",
  "Advaita",
  "Dhyana",
  "Mahatma",
  "Ardhanareshwara",
  "Saraswathi",
  "Surya Namaskara",
  "Chaitanya",
  "Lalita",
  "Ishwara",
  "Vijnana",
  "Achintya",
  "Bhagavatam",
  "Sankirtan",
  "Arya Samaj",
  "Tulsidas",
  "Nanak",
  "Sankhya",
  "Ashtanga Yoga",
  "Pashupata",
  "Lingam",
  "Tirtha",
  "Kshatriya",
  "Dvapara Yuga",
  "Gopala",
  "Ashtanga",
  "Jnana Vairagya",
  "Narayana",
  "Shabda",
  "Bhagwan",
  "Jnana Mudra",
  "Pinda",
  "Adi Shankaracharya",
  "Yagya",
  "Jagannath",
  "Sarva Dharma Sambhava",
  "Ganesha Chaturthi",
  "Rishikesh",
  "Narayana",
  "Giridhari",
  "Shyam",
  "Bhava Samadhi",
  "Aum Namah Shivaya",
  "Atma Jnana",
  "Swamiji",
  "Chakra",
  "Sarvam Khalvidam Brahma",
  "Rigveda",
  "Upasana",
  "Sant Kabir",
  "Aryavarta",
  "Varnashrama",
  "Ganga Aarti",
  "Ganesh Chaturthi",
  "Lingam",
  "Yajnopavita",
  "Rudra",
  "Linga",
  "Siddhi",
  "Kundalini",
  "Siddhi",
  "Ananda",
  "Giridhari",
  "Bhagwad Gita",
  "Bhagavan",
  "Bhagawan",
  "Katha",
  "Svadhyaya",
  "Gita",
  "Satchitananda",
  "Ardhanarishvara",
  "Sarvam Khalvidam Brahma",
  "Hare Krishna",
  "Ashtavakra",
  "Sankirtan",
  "Vijnana",
  "Vedanta",
  "Soma",
  "Vashishta",
  "Bharat",
  "Bharata",
  "Arya",
  "Guru Nanak",
  "Radha Krishna",
  "Kundalini Yoga",
  "Vedanta",
  "Gurukul",
  "Yugadharma",
  "Advaita",
  "Dhyana",
  "Mahatma",
  "Ardhanareshwara",
  "Saraswathi",
  "Surya Namaskara",
  "Chaitanya",
  "Lalita",
  "Ishwara",
  "Vijnana",
  "Achintya",
  "Bhagavatam",
  "Sankirtan",
  "Arya Samaj",
  "Tulsidas",
  "Nanak",
  "Sankhya",
  "Ashtanga Yoga",
  "Pashupata",
  "Lingam",
  "Tirtha",
  "Kshatriya",
  "Dvapara Yuga",
  "Gopala",
  "Ashtanga",
  "Jnana Vairagya",
  "Narayana",
  "Shabda",
  "Bhagwan",
  "Jnana Mudra",
  "Pinda",
  "Adi Shankaracharya",
  "Yagya",
  "Jagannath",
  "Sarva Dharma Sambhava",
  "Ganesha Chaturthi",
  "Rishikesh",
  "Narayana",
  "Giridhari",
  "Shyam",
  "Bhava Samadhi",
  "Aum Namah Shivaya",
  "Atma Jnana",
  "Swamiji",
  "Chakra",
  "Sarvam Khalvidam Brahma",
  "Rigveda",
  "Upasana",
  "Sant Kabir",
  "Aryavarta",
  "Varnashrama",
  "Ganga Aarti",
  "Ganesh Chaturthi",
  "Lingam",
  "Yajnopavita",
  "Rudra",
  "Linga",
  "Siddhi",
  "Kundalini",
  "Siddhi",
  "Ananda",
  "Giridhari",
  "Bhagwad Gita",
  "Bhagavan",
  "Bhagawan",
  "Katha",
  "Svadhyaya",
  "Gita",
  "Satchitananda",
  "Ardhanarishvara",
  "Sarvam Khalvidam Brahma",
  "Hare Krishna",
  "Ashtavakra",
  "Sankirtan",
  "Vijnana",
  "Vedanta",
  "Soma",
  "Vashishta",
  "Bharat",
  "Bharata",
  "Arya",
  "Guru Nanak",
  "Radha Krishna",
  "Kundalini Yoga",
  "Vedanta",
  "Gurukul",
  "Yugadharma",
  "Advaita",
  "Dhyana",
  "Mahatma",
  "Ardhanareshwara",
  "Saraswathi",
  "Surya Namaskara",
  "Chaitanya",
  "Lalita",
  "Ishwara",
  "Vijnana",
  "Achintya",
  "Bhagavatam",
  "Sankirtan",
  "Arya Samaj",
  "Tulsidas",
  "Nanak",
  "Sankhya",
  "Ashtanga Yoga",
  "Pashupata",
  "Lingam",
  "Tirtha",
  'Hinduphobia', 'Brahmanism', 'Bhakt', 'Cow piss drinker', 'Cow worshipper', 'Devil worshipper',
                 'Dothead',
                 'Dung worshipper',
                 'Dual loyalty',
                 'Exotic',
                 'Hindu fatalism',
                 'Heathen',
                 'Heritage Learner',
                 'Hindutvavadi',
                 'Idol Worshipper',
                 'Infidel', 'Kaffir', 'Malaun',
                 'Model Minority', 'Savarna', 'Unreached', 'Pajeet', 'Kshatriyas', 'Vaishyas', 'Sudras', 'Shaivism',
                 'Shaktism', 'Smartism', 'Shiva','diwali covid', 'navaratri', 'ram navami', 'ganesh chaturthi',
    'temple closures', 'online puja', 'virtual darshan', 'social distancing temples',
    'kumbh mela', 'religious events', 'pilgrimage restrictions', 'rituals lockdown',
    'prayers covid', 'virtual satsangs', 'hindu leaders covid', 'spiritual guidance',
    'mythological lessons', 'symbolism pandemic', 'resilience stories', 'hindu relief efforts',
    'seva pandemic', 'community support', 'karma pandemic', 'dharma challenges', 'reflections life death',
    'yoga lockdown', 'ayurveda immunity', 'wellness hindu philosophy', 'hindu diaspora covid',
    'cultural impact', 'contributions relief' ]


In [14]:
def exactMatch(txt,hinduTweets):
  for w in hinduismWords:
    f = 0
    if w .lower() in txt:
      hinduTweets.append(txt)
      f = 1
      return True
  return False


In [15]:
!gdown 1V8Q8Ody0KRk6H_38AVAL-EdYSEfhuIQV

Downloading...
From (uriginal): https://drive.google.com/uc?id=1V8Q8Ody0KRk6H_38AVAL-EdYSEfhuIQV
From (redirected): https://drive.google.com/uc?id=1V8Q8Ody0KRk6H_38AVAL-EdYSEfhuIQV&confirm=t&uuid=59cb76c7-9a9a-4f2e-ade8-e0c22ed2e19c
To: /kaggle/working/Tweets_Australia.csv
100%|█████████████████████████████████████████| 737M/737M [00:06<00:00, 109MB/s]


In [16]:
df_aus = pd.read_csv("/kaggle/working/Tweets_Australia.csv",index_col=0)

/tmp/ipykernel_19/2486228573.py:1: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_aus = pd.read_csv("/kaggle/working/Tweets_Australia.csv",index_col=0)


In [17]:
df_aus.head()

,created_at,id,text,user_location
11,Wed Dec 08 04:25:46 +0000 2021,1468436616263909376.0,RT @AaronSiriSG: Litigation update: \n\nFDA d...,"Iasi, Romania"
229,Wed Dec 08 04:25:57 +0000 2021,1468436661633630208.0,RT @covidbaseau: The TGA has approved the Mode...,Canberra
236,Wed Dec 08 04:26:06 +0000 2021,1468436701043314688.0,RT @SimoLove: Federal Health Minister Greg Hun...,"Wurundjeri Richmond, Melbourne"
250,Wed Dec 08 04:26:07 +0000 2021,1468436704755286016.0,@twityouse Are these the deniers that are wear...,"Victoria, Australia"
257,Wed Dec 08 04:26:07 +0000 2021,1468436706424545280.0,RT @drsimonegold: WOW: a preprint study for Th...,Perth WA


In [18]:
def prepro(df):
    global preprocess
    global contractions
    global contractionsWithAnotherInvertedComma
    #index_col = 0
    df.reset_index(inplace = True)
    df.dropna(subset = ["text"], inplace = True)
    df.drop_duplicates(subset = ["text"], inplace = True)
    pp_class = preprocess(df, contractions, contractionsWithAnotherInvertedComma)
    df['text'] = df['text'].progress_apply(lambda x : pp_class.preprocess_tweet(x))
    return df
    
    

In [19]:
df_aus = prepro(df_aus)

100%|██████████| 1630821/1630821 [7:50:25<00:00, 57.78it/s]


In [20]:
df_aus.to_csv("preproAus.csv")